### Data Scraping
Goal: use Python to scrape Leafly's database to use for qualitative data. Then encode into quantitative data.
Reference: https://medium.com/@aandrei_38387/mining-leafly-data-ae87c4b73856 

https://github.com/gugarosa/leaflyer

In [138]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
import time

In [139]:
#find list of strains
base_url = 'https://www.leafly.com/strains/?page=2'
response = requests.get(base_url)
soup = BeautifulSoup(response.text, "html.parser")
a = soup.findAll('a', {'data-testid': 'strain-card'})
# a['href']
for i in a:
    print(i['href'])
print(len(a))

/strains/wedding-cake
/strains/original-glue
/strains/runtz
/strains/gelato
/strains/ice-cream-cake
/strains/blue-dream
/strains/oreoz
/strains/apple-fritter
/strains/gary-payton
/strains/sour-diesel
/strains/zkittlez
/strains/jealousy
/strains/gsc
/strains/og-kush
/strains/lemon-cherry-gelato
/strains/biscotti
/strains/cereal-milk
/strains/mac-1
18


In [32]:
strains = ['og-kush'] #, 'blue-dream', 'sour-diesel','lemon-haze'

In [115]:
db=[]
for strain in (strains):
    obj={}
    print('now searching: ',base_url+strain)
    response = requests.get(base_url+strain)
    soup = BeautifulSoup(response.text, "html.parser")

    #find terpenes
    terpenes = soup.find('script',{'id': '__NEXT_DATA__'})
    json_data = json.loads(terpenes.contents[0])
    # print(json_data['props']['pageProps']['strain']['cannabinoids'])
    obj['slug'] = json_data['props']['pageProps']['strain']['slug']
    obj['id'] = json_data['props']['pageProps']['strain']['id']
    obj['rating'] = json_data['props']['pageProps']['strain']['averageRating']
    obj['terpenes'] = json_data['props']['pageProps']['strain']['terps']
    obj['symptoms'] = json_data['props']['pageProps']['strain']['symptoms']

    #find feelings
    feeling_data = soup.findAll("p", {'data-testid': 'item-name'})
    feelings = [i.text for i in feeling_data]
    obj['feelings'] = feelings

    #find helps_with in the form of [attribute, percentage] where percentage of people said it helped with attribute
    helps_with = soup.findAll("div",{'id': 'helps-with-section'})
    percentages = []
    attributes = []
    for div in helps_with:
        a = div.findAll('a', {'class': 'underline'})
        attributes.extend(a)
        b = div.findAll('span', {'class': 'font-bold'})
        percentages.extend(b)
    helps_with = [[attributes[i].text, percentages[i].text] for i in range(len(percentages))]
    obj['helps_with'] = helps_with

    db.append(obj)

now searching:  https://www.leafly.com/strains/og-kush


{'slug': 'og-kush',
 'id': 23,
 'rating': 4.273416350839199,
 'terpenes': {'caryophyllene': {'name': 'Caryophyllene',
   'description': 'Spicy, peppery terpene that may have anti-inflammatory benefits.',
   'score': 0.34774035106731},
  'humulene': {'name': 'Humulene',
   'description': 'Herbal terpene that may provide anti-inflammatory benefits.',
   'score': 0.109023651938524},
  'limonene': {'name': 'Limonene',
   'description': 'Citrus-scented terpene commonly believed to provide anxiety and stress relief.',
   'score': 0.389003874216203},
  'linalool': {'name': 'Linalool',
   'description': 'Floral terpene that is also found in lavender and commonly believed to promote relaxation.',
   'score': 0.139208380097903},
  'myrcene': {'name': 'Myrcene',
   'description': 'The most common terpene found in cannabis and has an earthy scent.',
   'score': 0.496960894080617},
  'ocimene': {'name': 'Ocimene',
   'description': 'Sweet, floral terpene that is commonly used in perfumes.',
   'sco